# Assessing the suitability of a dataset for deep learning
This notebook shows two examples of applying a fastai deep learning model to a tabular dataset. The goal of the notebook is to show a contrast between a unsuccessful application and a successful application.

In [49]:
# imports for notebook boilerplate
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.tabular.all import *

In [50]:
# set up the notebook for fast.ai
fastbook.setup_book()

In [51]:
# imports specifically needed for this notebook
! pip install pandas_datareader
import numpy as np
import pandas as pd
import os
import yaml
# For reading stock data
from pandas_datareader.data import DataReader

# For time stamps
from datetime import datetime


# Ingest the dataset
Uset the DataReader API to get a stock price dataset for a stock.

In [52]:
# Set up start and end times for the data load - a year previous
end_time = datetime.now()
start_time = datetime(end_time.year - 1, end_time.month, end_time.day)


In [53]:
# Use the DataReader API to get the stock prices for AZN (AstraZeneca)
df = DataReader('AZN', 'stooq')
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2021-07-09,59.12,59.910,59.070,59.63,6173577
2021-07-08,59.45,59.670,58.830,59.26,6894951
2021-07-07,59.76,59.820,59.295,59.76,6434879
2021-07-06,60.41,60.555,59.870,59.90,6893637
2021-07-02,60.26,60.875,60.065,60.79,5198428


In [54]:
df.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2016-07-18,25.531,25.658,25.503,25.591,6983834
2016-07-15,25.480,25.513,25.231,25.330,8774132
2016-07-14,25.531,25.550,25.263,25.415,11345005
2016-07-13,25.389,25.528,25.255,25.263,11485284
2016-07-12,25.021,25.389,24.987,25.299,13560891


In [55]:
df.shape

(1258, 5)

In [56]:
# check for missing values
count = df.isna().sum()
df_missing = (pd.concat([count.rename('missing_count'),
                     count.div(len(df))
                          .rename('missing_ratio')],axis = 1)
             .loc[count.ne(0)])

In [57]:
# check for missing values
df_missing

,missing_count,missing_ratio


# Build and train the first model
For the fist model, use fastai defaults for everything.

In [58]:
dep_var = 'Close'
# define columns that are continuous / categorical
cont,cat = cont_cat_split(df, 1, dep_var=dep_var) 
print("continuous columns are: ",cont)
print("categorical columns are: ",cat)

continuous columns are:  ['Open', 'High', 'Low', 'Volume']
categorical columns are:  []


In [59]:
procs = [Normalize]
dls = TabularDataLoaders.from_df(df,procs= procs, 
                                 cat_names= cat, cont_names = cont, 
                                 y_names = dep_var, 
                                 valid_idx=list(range((df.shape[0]-50),df.shape[0])), bs=64)

In [60]:
dls.valid.show_batch()

,Open,High,Low,Volume,Close
0,28.834000,28.903000,28.503000,4.041644e+06,28.554001
1,28.523000,28.632000,28.395001,3.924227e+06,28.443001
2,28.954001,28.979999,28.580000,6.074469e+06,28.700001
3,28.165000,28.699000,28.003001,7.098700e+06,28.563999
4,27.927999,28.232999,27.859000,8.215522e+06,28.087999
5,27.884001,27.991000,27.563000,6.874591e+06,27.783001
6,27.605000,28.292000,27.554001,4.246458e+06,28.207001
7,27.876000,27.876000,27.312000,5.263818e+06,27.323000
8,27.859000,28.070999,27.850000,7.152325e+06,28.039000
9,28.063000,28.180001,27.970000,5.288114e+06,28.157000


In [61]:
# define and fit the model
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,1562.704834,698.903931,0.000000,00:00
1,1393.395386,487.360260,0.000000,00:00
2,1241.338989,416.841797,0.000000,00:00


# Build and train the second model
Revise the model:
- define a new target column to act as a categorical dependent variable (replacing the continuous dependent variable from the first model)
- explicitly select a subset of columns to train the model rather taking the set provided by default by cont_cat_split()

In [62]:
def get_target(value,threshold):
    '''return based on whether the input value is greater than or less than input threshold'''
    if value <= threshold:
        return_value = "0"
    else:
        return_value = "1"
    return(return_value)
    

In [63]:
threshold = df['Close'].mean()
df['target'] = df['Close'].apply(lambda x: get_target(x,threshold))

In [64]:
df['target'].value_counts()

0    722
1    536
Name: target, dtype: int64

In [65]:
dep_var = 'target'

In [66]:
cont = ['High', 'Low', 'Open', 'Volume']

In [67]:
dls = TabularDataLoaders.from_df(df,procs= procs, 
                                 cat_names= cat, cont_names = cont, 
                                 y_names = dep_var, 
                                 valid_idx=list(range((df.shape[0]-50),df.shape[0])), bs=64)
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy,time
0,0.645138,0.602366,0.980000,00:00
1,0.520773,0.289147,1.000000,00:00
2,0.383414,0.065214,1.000000,00:00
3,0.278487,0.009715,1.000000,00:00
4,0.208815,0.002345,1.000000,00:00
5,0.157652,0.000606,1.000000,00:00
6,0.123080,0.000578,1.000000,00:00
7,0.114773,0.000280,1.000000,00:00
8,0.098626,0.000171,1.000000,00:00
9,0.090189,0.000279,1.000000,00:00


In [68]:
df_results = learn.show_results(max_n=50)

,High,Low,Open,Volume,target,target_pred
0,-1.031832,-1.022653,-1.036134,-0.483110,0.0,0.0
1,-1.082339,-1.070244,-1.084488,0.344353,0.0,0.0
2,-1.328620,-1.328356,-1.313748,1.333216,0.0,0.0
3,-1.020989,-1.054979,-1.054353,0.331705,0.0,0.0
4,-1.054302,-1.050589,-1.061049,-0.653724,0.0,0.0
5,-1.063289,-1.048993,-1.057701,-0.037859,0.0,0.0
6,-1.271470,-1.278969,-1.260864,1.004372,0.0,0.0
7,-1.027534,-1.015868,-1.019097,-0.417010,0.0,0.0
8,-1.052055,-1.073138,-1.057701,0.002419,0.0,0.0
9,-1.008582,-1.003097,-1.012696,-0.350954,0.0,0.0
